PIR_3 : Segmentation de tumeurs par YOLOv8 sur la database BRATS

_______________________________________________________________________________________________________________________________________________________________

1) __Intro__ : 

Nous souhaitons créer un model YOLOv8 à la segmentation de tumeurs BRATS. Pour cela nous avons utilisé la database BRATS. Npus partons donc de données compressées sous formats '.nii'. Cette data est composé de scans 3D cérébraux de 393 patients. Chaque scan est décomposé en 2 fichier '.nii' correspondant à la vérité de terrain (scan 3D segmenté à la main en plusieurs labels => niveau de gris entre 0 et 5) et au cliché à proprement parlé (coupe du cerveau dans les 3 axes).

La database BRATS est disponible jusqu'au 05/06/2023 à l'adresse suivante : https://filesender.renater.fr/?s=download&token=0f6680d2-be1b-42b5-9153-227dda42896b

Un dataset prêt pour l'apprentissage et la prediction YOLOv8 est disponible jusqu'au 05/06/2023 à l'adresse suivante : https://filesender.renater.fr/?s=download&token=e1b70e25-8e53-4e14-93ec-e811caa2f133

Les résultats d'un model entrainé est disponible jusqu'au 05/06/2023 à l'adresse suivante : https://filesender.renater.fr/?s=download&token=c756029c-ebf1-4704-be7f-edb8d5094f1a

YOLOv8 est un réseau neuronal profond édité par Ultralytics. Toute la doc peut être directement retrouvez sur le site : https://docs.ultralytics.com/tasks/detect/

La biblithéque implementant YOLOv8 est sur : https://github.com/ultralytics/ultralytics 

Le l'environnement correspondant ce jupyter est présent sur le github suivant : https://github.com/ElMagicarp/PIR_3_YOLO/tree/main/Brats_with_YOLOv8

__ATTENTION__ : nous utilisons ici Pytorch en mode 'cpu' pour faire tourner le réseau

_______________________________________________________________________________________________________________________________________________________________

__Préparation des imports / pip__

In [ ]:
import os
import sys

os.system('pip install -r requirements.txt')

In [ ]:
import yoloTools
import SLICER as slc
import FORMATAGE as frm
from picsellia.types.enums import AnnotationFileType
from ultralytics import YOLO
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import sys
from PIL import Image
from matplotlib import pyplot as plt
import glob
import torch


_______________________________________________________________________________________________________________________________________________________________

2) __Formatage de la database__ :

Dans cette implementation, nous utiliserons le formatage en répertoires suivant (contrainte ultralytics + yolotools):

```
-- datasets  +-- raw  +-- test  +-- images
             |        |         |
             |        |         +-- labels
             |        |
             |        +-- train +-- images
             |        |         |
             |        |         +-- labels
             |        |
             |        +-- val   +-- images
             |                  |
             |                  +-- labels
             |
             +-- refined  +-- test  +-- images
                          |         |
                          |         +-- labels
                          |
                          +-- train +-- images
                          |         |
                          |         +-- labels
                          |
                          +-- val   +-- images
                                    |
                                    +-- labels
```

__Pour constituer le dataset :__
 1) Télécharger l'archive BRATS
 2) Extraire l'archive dans le répertoire datasets
 3) Déplacer tout les fichiers compressés .nii.gz dans répertoire datasets
 4) Supprimer le répertoire vide issu de l'archive BRATS
 5) Executer les scripts suivants ce markdown jusqu'à la prochaine section

__a) déclaration des config d'extraction des données (slice en png et réparti dans les bons fichiers)__

In [ ]:
path_vers_datasets_raw = '../datasets/raw' #chemin relatif vers le répertoire dataset/raw
seuil = 1 #seuil disciminant pour la sélection des coupes (int entre 1 et 100 => rapport nb pixels segementés/nb pixels)
nb_coupe_par_scan = 5 #nb de coupes max extraites par fichier .nii.gz 

__facultatif) test du dataset pour connaitre seuil max pour un nb de coupes par scan__

In [ ]:
print("#_____________TEST_____________#")
slc.slicer_main(path_vers_datasets_raw, nb_coupe_par_scan,testSeuil=True)

__b) slice(.nii.gz -> png) + distribution dans les bons répertoires (refined/..) avec la répartition suivante:__

train = 64% dataset ; val = 16% dataset ; test = 20% dataset

Rq :  les patients ne peuvent être présent que dans un seul des sous répertoires

In [ ]:
print('#_____________SELECTION_____________#')
slc.slicer_main(path_vers_datasets_raw, seuil, nb_coupe_par_scan)

__d) déclaration des config pour le formatage des données__

__ATTENTION__ : à cause de cv2, la segmentation ne se fait qu'en binaire, on ramene alors touts les labels à la même valeur ce qui nous empêche de les différencier

__ATTENTION__ : si vous souhaitez changer de format ('bbox' ou 'seg') aprés avoir déjà formaté le dataset, vous devez repartir de la commande 2.b) en modifiant la variable 'mode' de 2.d) 

In [ ]:
path_vers_datasets_refined = '../datasets/refined' #chemin relatif vers le répertoire dataset/refined
mode = 'seg' #indique le format souhaité 'seg' ou 'bbox' => 'bbox' pour bounding box et 'seg' pour segmentation
resize = 128 # résolution de l'image carrée de sortie => reformatage en image carrée multiple de 32 à cause de YOLOv8 (ou changer 1ére couche du réseau)

__c) formatage du dataset avec génération du json format COCO et des data.yaml__

In [ ]:
frm.formatage_main(path_vers_datasets_refined,mode,resize)

_______________________________________________________________________________________________________________________________________________________________

3) __Implémentation de YOLOv8 : entrainement__

__ATTENTION__ : 'le fichier data.yaml généré à l'opération précédente se situe dans le répertoire datasets/refined, il faut modifier le chemin d'accés dans la déclaration du model ou renomer puis déplacer data.yaml dans le fichier code

In [ ]:
#BRATS
#model = YOLO('/home/jovyan/workspace/Yolov8/runs/detect/BRATS2/weights/best.pt'
type_reseau = 'seg'
tache = ''
dirSortie =''

if type_reseau == 'bbox':
    model = YOLO('yolov8l.yaml') # yolov8l.yaml pour partir d'un modéle vierge, sinon indiquer le chemin vers model(.pt) pré-entrainé
    yaml = 'dataBBOX.yaml'
    tache = 'detect'
    dirSortie ='BRATS_Bbox'
    
elif type_reseau == 'seg':
    model = YOLO('yolov8l-seg.yaml') # yolov8l-seg.yaml pour partir d'un modéle vierge, sinon indiquer le chemin vers model(.pt) pré-entrainé
    yaml = 'dataSEG.yaml'
    tache = 'segment'
    dirSortie ='BRATS_Seg'

In [ ]:
#train BRATS
results = model.train(

   # il y a d'autres parametres possible => cf. site ultralytics yolov8
   data=yaml,
   task=tache,
   imgsz=128,
   epochs=30,
   batch=10,
   name='BRATS_Seg', # nom du répertoire de sortie
   workers = 2,
)

En sortie, yolov8 crée un nouveau répertoire dans code/runs/segment (pour segmentation) ou code/runs/detect (pour bounding box). Dans ce répertoire on retrouve le model entrainé (./weights : best.pt pour le model avec le meilleurs dice et last.pt le dernier model généré), un .csv recapitulant les stats tout au long des epochs, des graphs et des resultats de prédiction (seg ou bbox)

_______________________________________________________________________________________________________________________________________________________________


4) __Implémentation de YOLOv8 : prediction__

__ATTENTION__ : pas testé, mais doit marcher

In [ ]:
#predict BRATS
results = model.predict(source = yaml,
                         save=True,
                         show=True,
                         imgsz=128,
                         conf=0.5)